# Region of Contraction Estimation

In [ ]:
# python libraries
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import solve_ivp

# pydrake imports
from pydrake.all import (MathematicalProgram, Variables,
                         Solve, RealContinuousLyapunovEquation)
import pydrake.symbolic as sym

# underactuated imports
from underactuated import plot_2d_phase_portrait

# increase default size matplotlib figures
from matplotlib import rcParams
rcParams['figure.figsize'] = (6, 6)

In [ ]:
%env MOSEKLM_LICENSE_FILE = "mosek.lic"
from pydrake.solvers.mosek import MosekSolver
from pydrake.solvers.snopt import SnoptSolver
from pydrake.solvers.ipopt import IpoptSolver

env: MOSEKLM_LICENSE_FILE="mosek.lic"


In [ ]:
def van_der_pol_ball():
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(2, "x")
    f = [- x[1], x[0] + (x[0]**2 - 1) * x[1]]

    M_poly_deg = 4
    M_11 = prog.NewFreePolynomial(Variables(x), M_poly_deg).ToExpression()
    M_12 = prog.NewFreePolynomial(Variables(x), M_poly_deg).ToExpression()
    M_22 = prog.NewFreePolynomial(Variables(x), M_poly_deg).ToExpression()

    M = np.array([[M_11, M_12], [M_12, M_22]])
    M_11_dot = M_11.Jacobian(x).dot(f)
    M_12_dot = M_12.Jacobian(x).dot(f)
    M_22_dot = M_22.Jacobian(x).dot(f)
    M_dot = np.array([[M_11_dot, M_12_dot], [M_12_dot, M_22_dot]])

    dfdx = [[0, -1], [1 + 2*x[0]*x[1], x[0]**2 - 1]]

    # lam = prog.NewContinuousVariables(1, "lam")[0]
    lam = 0
    R = (M.dot(dfdx)).transpose() + M.dot(dfdx) + M_dot + 2*lam*M

    y = prog.NewIndeterminates(2, "y")
    rho = prog.NewContinuousVariables(1, "rho")[0]
    # l = prog.NewFreePolynomial(Variables(x), 2).ToExpression()

    M_rho = M + (x.dot(x) - rho)*np.eye(2)
    P_M = y.dot(M_rho).dot(y)
    R_rho = -R + (x.dot(x) - rho)*np.eye(2)
    P_R = y.dot(R_rho).dot(y)

    eps = 1e-5
    
    prog.AddSosConstraint(P_M - eps*y.dot(y))
    prog.AddSosConstraint(P_R - eps*y.dot(y))
    prog.AddConstraint(rho >= 0)

    # print(rho)
    prog.AddLinearCost(-rho)

    # print(prog)
    solver = MosekSolver()
    result = solver.Solve(prog)
    if result.is_success:
        # print("success")
        # print(result)
        print(result.GetSolution(M))
        # print(result.GetSolution(R))
        return result.GetSolution(M), result.GetSolution(rho)
    else:
        print("Could not find a contraction metric")

In [ ]:
van_der_pol_ball()

[[<Expression "(1.4978995949598728 - 8.9921299181879e-19 * x(0) + 6.0532791304402332e-18 * x(1) + 0.012671789600241666 * (x(0) * x(1)) - 5.7602710143115141e-19 * (x(0) * pow(x(1), 2)) - 2.1343407336767106e-06 * (x(0) * pow(x(1), 3)) + 6.162610656806576e-21 * (pow(x(0), 2) * x(1)) - 0.0024275139625661792 * (pow(x(0), 2) * pow(x(1), 2)) - 0.9992434147794248 * pow(x(0), 2) - 1.3594501208445591e-18 * pow(x(0), 3) + 0.44873043764376391 * pow(x(0), 4) - 0.18116751300020276 * pow(x(1), 2) + 7.1490592623836078e-21 * pow(x(1), 3) + 3.2488378831303897e-06 * pow(x(1), 4))">
  <Expression "(-0.4044869214327671 - 2.4436342358248764e-18 * x(0) + 4.5466420155699369e-18 * x(1) + 0.057109978759730229 * (x(0) * x(1)) - 2.8893549724847252e-20 * (x(0) * pow(x(1), 2)) - 4.0743767286253312e-06 * (x(0) * pow(x(1), 3)) - 1.6027490047084984e-20 * (pow(x(0), 2) * x(1)) + 1.611109631924947e-05 * (pow(x(0), 2) * pow(x(1), 2)) + 3.2322678834905096e-06 * (pow(x(0), 3) * x(1)) + 0.44878123817702426 * pow(x(0), 2) + 

(array([[<Expression "(1.4978995949598728 - 8.9921299181879e-19 * x(0) + 6.0532791304402332e-18 * x(1) + 0.012671789600241666 * (x(0) * x(1)) - 5.7602710143115141e-19 * (x(0) * pow(x(1), 2)) - 2.1343407336767106e-06 * (x(0) * pow(x(1), 3)) + 6.162610656806576e-21 * (pow(x(0), 2) * x(1)) - 0.0024275139625661792 * (pow(x(0), 2) * pow(x(1), 2)) - 0.9992434147794248 * pow(x(0), 2) - 1.3594501208445591e-18 * pow(x(0), 3) + 0.44873043764376391 * pow(x(0), 4) - 0.18116751300020276 * pow(x(1), 2) + 7.1490592623836078e-21 * pow(x(1), 3) + 3.2488378831303897e-06 * pow(x(1), 4))">,
         <Expression "(-0.4044869214327671 - 2.4436342358248764e-18 * x(0) + 4.5466420155699369e-18 * x(1) + 0.057109978759730229 * (x(0) * x(1)) - 2.8893549724847252e-20 * (x(0) * pow(x(1), 2)) - 4.0743767286253312e-06 * (x(0) * pow(x(1), 3)) - 1.6027490047084984e-20 * (pow(x(0), 2) * x(1)) + 1.611109631924947e-05 * (pow(x(0), 2) * pow(x(1), 2)) + 3.2322678834905096e-06 * (pow(x(0), 3) * x(1)) + 0.44878123817702426 * 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=21c3e526-6092-407b-92b0-f6897737c93c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>